Classify images into 10 categories:
airplane, car, bird, cat, deer, dog, frog, horse, ship, truck"

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

c:\Users\pc\miniconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 51s 0us/step


c:\Users\pc\miniconda3\Lib\site-packages\keras\src\datasets\cifar.py:18: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


Each image is 32×32×3

32 → width

32 → height

3 → RGB (Red, Green, Blue)

Labels are numbers from 0 to 9

In [3]:
x_train = x_train / 255.0
x_test = x_test / 255.0

Original pixel values: 0 – 255

After dividing: 0 – 1

Helps:

Faster learning

Stable gradients

Better accuracy

👉 CNNs always prefer small, consistent values.

In [ ]:
model = models.Sequential([
    
    # Convolution Block 1
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    #32 filters → model learns 32 different features
    #3×3 filter size → looks at small image regions
    #Slides over image → detects edges, colors, textures
    #Same filter used everywhere → weight sharing
    #ReLU → removes useless negative values
    #This is where CNN actually “sees” the image.

    layers.MaxPooling2D((2,2)),
    #Takes maximum value from each 2×2 region
    #Reduces image size
    #Keeps strongest features
    #Like summarizing important information.
    
    # Convolution Block 2
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    # Flatten and Dense Layers
    layers.Flatten(),
    #Converts feature maps into 1D vector
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])


c:\Users\pc\miniconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
model.fit(x_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - accuracy: 0.4440 - loss: 1.5341
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.5844 - loss: 1.1894
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.6336 - loss: 1.0535
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.6653 - loss: 0.9623
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 19ms/step - accuracy: 0.6891 - loss: 0.8973
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.7073 - loss: 0.8419
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.7256 - loss: 0.7928
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 19ms/step - accuracy: 0.7407 - loss: 0.7491
Epoch 9/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.7555 - loss: 0.7068
Epoch 10/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.7675 - loss: 0.6711


In [7]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Accuracy:", test_acc)


313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6854 - loss: 0.9484
Test Accuracy: 0.6854000091552734
